In [ ]:
import exputils.font

exputils.font.font_use_tex()

In [ ]:
import numpy
import scipy.optimize
import sympy

In [ ]:
import exputils.figure

fh = exputils.figure.FigureHandler(save=True, fmt="Figure-{ctr:03}.pgf")

In [ ]:
def unif_approx_disc_coef(func, ctrl):
    
    n = ctrl.size - 1
    
    lst = [ctrl**i for i in range(n)] + [(-1)**numpy.arange(n+1)]
    a = numpy.vstack(lst).transpose()
    
    b = func(ctrl)
    
    x = numpy.linalg.solve(a, b)
    
    c, d = x[:n], x[n]
    
    return c, d

In [ ]:
def coef_poly(coefficient):
    def poly(x):
        return numpy.sum(c * x**i for i, c in enumerate(coefficient))
    return poly

In [ ]:
def plot_func(func, left=-1.0, right=1.0, num=1000, fh=fh, **kwargs):
    x = numpy.linspace(left, right, num)
    y = func(x)
    fh.ax.plot(x, y, **kwargs)

In [ ]:
def cheb_exts(n, left=-1.0, right=1.0):
    x = numpy.cos(numpy.linspace(0.0, numpy.pi, n+1))[::-1]
    x = x * (right - left) / 2.0 + (right + left) / 2.0
    return x

In [ ]:
def f(t):
    return numpy.abs(t**3)

In [ ]:
n = 6

its = 5

eps = 1.0e-4

In [ ]:
fh.new_fig()
fh.new_ax()

x = cheb_exts(n)

plot_func(f, fh=fh, label="$f$")

for i in range(its):
    c, d = unif_approx_disc_coef(f, x)
    
    p = coef_poly(c)
    
    plot_func(p, fh=fh, label="$p_{}$".format(i+1))

    xi = [-1.0] + [scipy.optimize.brentq(
        lambda t: p(t) - f(t),
        x[i], x[i+1]
    ) for i in range(n)] + [1.0]

    sig = (-1) ** numpy.arange(n+1)
    if numpy.signbit(d):
        sig *= -1
    
    opt = [scipy.optimize.minimize(
        lambda t: sig[i] * (p(t) - f(t)),
        (xi[i] + xi[i+1]) / 2.0,
        method="L-BFGS-B", ###
        bounds=((xi[i], xi[i+1]),)
    ) for i in range(n+1)]
    
    err_list = [float(o["fun"]) for o in opt]
    err = max(err_list) - min(err_list)
    print("Step {0}, error {1}".format(i, err))
    if err < eps:
        break
    
    x = numpy.hstack([o["x"] for o in opt])

fh.set_box(-1.1, 1.1, -0.1, 1.1, grid=True)

fh.ax.legend()

fh.disp_fig()
fh.close_fig()

In [ ]:
fh.new_fig()
fh.new_ax()

plot_func(lambda t: p(t) - f(t), fh=fh, label="$ p_{} - f $".format(i+1))

fh.set_box(-1.1, 1.1, -0.015, 0.015, grid=True)

fh.ax.legend()

fh.disp_fig()
fh.close_fig()

In [ ]:
print(", ".join([str(p) for p in x]))

In [ ]:
sympy.init_printing()

x = sympy.Symbol("x")

poly = sum([x**deg * coe for deg, coe in enumerate(c)])

In [ ]:
poly